In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import numpy as np

# Initialize the WebDriver
driver = webdriver.Chrome("C:\\Users\\my pc\\Downloads\\chromedriver-win64\\chromedriver")

# List of US states
states = ["California", "Texas", "Florida", "New York", "Alabama", "Alaska", "Arizona"]  

# Dates for the winter season (example: December 15, 2024, to December 16, 2024)
checkin_date = "2024-12-15"
checkout_date = "2024-12-16"

hotel_data = []
for state in states:
    url = "https://www.booking.com"
    driver.get(url)

    # Enter the state in the search box
    search_box = driver.find_element_by_xpath("/html/body/div[3]/div[2]/div/form/div[1]/div[1]/div/div/div[1]/div/div/input")
    search_box.clear()
    time.sleep(1)
    search_box.send_keys(state)
    # Enter check-in and check-out dates

    driver.find_element_by_xpath("/html/body/div[3]/div[2]/div/form/div[1]/div[2]/div/div/button[1]").click()
    driver.find_element_by_xpath("/html/body/div[3]/div[2]/div/form/div[1]/div[2]/div/div[2]/div/nav/div[2]/div/div[1]/div/div[2]/table/tbody/tr[3]/td[7]/span").click()
    driver.find_element_by_xpath("/html/body/div[3]/div[2]/div/form/div[1]/div[2]/div/div[2]/div/nav/div[2]/div/div[1]/div/div[2]/table/tbody/tr[4]/td[1]/span").click()   
    # Submit the search
    search_box.send_keys(Keys.RETURN)
    time.sleep(10)  # Wait for the results to load
    
    for _ in range(10):
        driver.execute_script("window.scrollBy(0,1000)")
        time.sleep(1)
    # Scrape hotel prices
    
    prices = driver.find_elements(By.CSS_SELECTOR, "span[data-testid='price-and-discounted-price']")
    #prices=driver.find_elements_by_id("")
    for price in prices:
            price_text = price.text.replace('£', '').strip()
            if price_text:
                hotel_data.append({
                    'state': state,
                    'price': float(price_text)
                })
    
    

    


In [108]:
# Convert to DataFrame
df = pd.DataFrame(hotel_data)

In [110]:
df

,state,price
0,California,133.0
1,California,70.0
2,California,301.0
3,California,180.0
4,California,145.0
...,...,...
249,Arizona,74.0
250,Arizona,154.0
251,Arizona,319.0
252,Arizona,70.0


In [113]:


df = pd.DataFrame(hotel_data)

#Clean the price data
df['price'] = df['price'].astype(float)

# Calculate average price per state
average_prices = df.groupby('state')['price'].mean().reset_index()

# Rank states by average price
ranked_states = average_prices.sort_values(by='price').reset_index(drop=True)

# Save the results to a CSV file
ranked_states.to_csv('us_states_hotel_prices_winter.csv', index=False)

# Close the driver
driver.quit()

print(ranked_states)

        state       price
0     Alabama   86.538462
1      Alaska  112.180000
2       Texas  130.400000
3     Arizona  141.686275
4    New York  172.400000
5  California  177.500000
6     Florida  191.230769
